# Analyzing Flight Price Differences

T11: Hanze Zhang (hanzez) & Yi-Hsueh Yang (yihsuehy)
Video Link: https://cmu.zoom.us/rec/share/maqX_iARNdnfQelFxzxco8NExJi8CNjE1RlKLdsLWofaf0byKxHSDI0n2wdQS9UI.2kuXCz4kN7ZfsaA_?startTime=1679015619000
Passcode: 8FTg?#1p

Data Source: https://github.com/dilwong/FlightPrices

https://www.kaggle.com/datasets/dilwong/flightprices

In [ ]:
# These two chunks are meant to import data from kaggle, due to its enormous size we choose this import method to relax the loading pressure for our local machines.
'''! pip install -q kaggle
from google.colab import files
files.upload()'''

In [ ]:
 '''! mkdir ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json
 ! kaggle datasets list
 !kaggle datasets download -d dilwong/flightprices
 !unzip flightprices.zip'''

In [ ]:
import pandas as pd
import altair as alt
import datetime
import re

# Disable max row limitation by Altair
alt.data_transformers.disable_max_rows()
# Disable limitation on the width of dataframe columns
pd.set_option('display.max_colwidth', None)

In [ ]:
# mount google drive to load in data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In this block of code, we are dealing with the size of our original dataset. It is a huge dataset with 30G+ of data, so we break it into smaller chunks of datasets by filtering out some unnecessary columns and limit the time duration to around 1 and a half month per dataset.

In [ ]:
'''chunksize = 1000

# Specify the columns to read from the CSV file
columns = ['searchDate', 'flightDate', 'startingAirport', 'destinationAirport', 'fareBasisCode', 'travelDuration', 
           'elapsedDays', 'isBasicEconomy', 'isRefundable', 'isNonStop', 'baseFare', 'totalFare', 'seatsRemaining', 
           'totalTravelDistance', 'segmentsDepartureTimeRaw', 'segmentsAirlineCode', 'segmentsEquipmentDescription','segmentsCabinCode']

column2 = ['searchDate', 'flightDate', 'totalFare']

# Create an empty list to store the filtered chunks
filtered_chunks = []

# Loop through the CSV file in chunks
for chunk in pd.read_csv('itineraries.csv', chunksize=chunksize, usecols=column2):
    # Filter the chunk based on a condition
    filtered_chunk = chunk[chunk['flightDate'] >= '2022-09-15']

    # Append the filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate the filtered chunks into a single DataFrame
filtered_df = pd.concat(filtered_chunks)

filtered_df = pd.read_csv('itineraries.csv', usecols=column2)

# save the filtered data to a csv file
filtered_df.to_csv('/content/drive/My Drive/Alltime_Data.csv', index=False)

print(filtered_df.shape)'''

## Analysis Starts Here

In [ ]:
april_may_df = pd.read_csv('drive/My Drive/Data_Science_Big_Data_T11/April_May_Data.csv')
april_may_df = april_may_df.drop(['segmentsEquipmentDescription', 
                                  'elapsedDays', 'baseFare'], axis=1)

april_may_df.head(10)

,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsAirlineCode,segmentsCabinCode
0,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,False,False,True,248.6,9,947.0,2022-04-17T12:57:00.000-04:00,DL,coach
1,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,False,False,True,248.6,4,947.0,2022-04-17T06:30:00.000-04:00,DL,coach
2,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,False,False,True,248.6,9,947.0,2022-04-17T11:35:00.000-04:00,DL,coach
3,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,False,False,True,248.6,8,947.0,2022-04-17T13:59:00.000-04:00,DL,coach
4,2022-04-16,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,False,False,True,248.6,9,947.0,2022-04-17T09:59:00.000-04:00,DL,coach
5,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,False,False,True,248.6,7,947.0,2022-04-17T10:45:00.000-04:00,B6,coach
6,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT4H12M,False,False,False,251.1,3,956.0,2022-04-17T08:20:00.000-04:00||2022-04-17T10:27:00.000-04:00,AA||AA,coach||coach
7,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H18M,False,False,False,251.1,3,956.0,2022-04-17T08:20:00.000-04:00||2022-04-17T11:20:00.000-04:00,AA||AA,coach||coach
8,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT5H32M,False,False,False,251.1,7,956.0,2022-04-17T07:00:00.000-04:00||2022-04-17T10:27:00.000-04:00,AA||AA,coach||coach
9,2022-04-16,2022-04-17,ATL,BOS,V0AJZNN1,PT6H38M,False,False,False,251.1,7,956.0,2022-04-17T07:00:00.000-04:00||2022-04-17T11:20:00.000-04:00,AA||AA,coach||coach


In [ ]:
def to_time(date_time):
    return '||'.join([pd.to_datetime(date).strftime('%H:%M:%S') for date in date_time.split('||')])
april_may_df['departure_time'] = april_may_df['segmentsDepartureTimeRaw'].apply(lambda x : to_time(x))
april_may_df = april_may_df.drop('segmentsDepartureTimeRaw', axis = 1)

1. Compare Mean flight price in different date
2. Compare Mean flight price in different days in a week 
3. Compare mean flight price in times in a day
4. compare flight price between airports in the same city
5. the relationship between price and the remaining seats of the same flight 


# Compare Mean flight price in different days in a week

In [ ]:
df = april_may_df.copy()

df['flightDate'] = pd.to_datetime(df['flightDate'])
df['searchDate'] = pd.to_datetime(df['searchDate'])

date_to_fare = pd.DataFrame(df.groupby('flightDate')['totalFare'].mean().reset_index())

In [ ]:
df['daysinweek'] = df['flightDate'].apply(lambda x: x.date().weekday())
df['search_daysinweek'] = df['searchDate'].apply(lambda x: x.date().weekday())

daysinweek_to_fare = pd.DataFrame(df.groupby('daysinweek')['totalFare'].mean().reset_index())
search_daysinweek_to_fare = pd.DataFrame(df.groupby('search_daysinweek')['totalFare'].mean().reset_index())

daysinweek_to_fare['type'] = ['schedule','schedule','schedule','schedule','schedule','schedule','schedule' ]
search_daysinweek_to_fare['type'] = ['search','search','search','search','search','search','search' ]

search_and_flight = pd.concat([search_daysinweek_to_fare, daysinweek_to_fare])

weekday_mapping = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
search_and_flight['weekday'] = daysinweek_to_fare['daysinweek'].map(weekday_mapping)

search_and_flight

,search_daysinweek,totalFare,type,daysinweek,weekday
0,0.0,348.536190,search,NaN,Mon
1,1.0,356.954535,search,NaN,Tue
2,2.0,358.033568,search,NaN,Wed
3,3.0,359.098796,search,NaN,Thu
4,4.0,354.752974,search,NaN,Fri
5,5.0,356.717495,search,NaN,Sat
6,6.0,345.487025,search,NaN,Sun
0,NaN,373.910616,schedule,0.0,Mon
1,NaN,306.309631,schedule,1.0,Tue
2,NaN,310.695726,schedule,2.0,Wed


In [ ]:
#0: Monday, 6: Sunday

alt.Chart(search_and_flight).mark_line().transform_fold(
    fold=['search', 'schedule'], 
).encode(
    x=alt.X('weekday:N', sort=['Mon','Tue','Wed','Thu','Fri','Sat','Sun']),
    y='totalFare:Q',
    color='type'
).properties(
    title = 'Mean Flight Price and Search Flight Price by day of week'
).interactive()

alt.Chart(...)

# Compare flight price between airports in the same city

Airports:
 DFW, DEN, ORD, LAX, CLT, MIA, JFK, EWR, SFO, DTW, BOS, PHL, LGA, IAD, OAK
*   ATL: Hartsfield-Jackson Atlanta International Airport
*   DFW: Dallas/Fort Worth International Airport
*   ORD: Chicago O'Hare International Airport
*   LAX: Los Angeles International Airport
*   CLT: Charlotte Douglas International Airport
*   MIA: Miami International Airport
*   JFK: John F. Kennedy International Airport (New York)
*   EWR: Newark Liberty International Airport (New York and New Jersey)
*   SFO: San Francisco International Airport
*   DTW: Detroit Metropolitan Wayne County Airport
*   BOS: Boston Logan International Airport
*   PHL: Philadelphia International Airport
*   LGA: LaGuardia Airport (New York)
*   IAD: Dulles International Airport
*   OAK: Oakland International Airport

Airports around New York City: JFK, EWR, LGA  
Airports around San Francisco: OAK, SFO


In [ ]:
# Filter out every tickets that flight to airports around NYC
new_york_df = april_may_df[(april_may_df['destinationAirport'] == 'JFK') | 
                           (april_may_df['destinationAirport'] == 'EWR') | 
                           (april_may_df['destinationAirport'] == 'LGA')].copy()

# Filter out every tickets that flight to airports around San Francisco
sf_df = april_may_df[(april_may_df['destinationAirport'] == 'OAK') | 
                     (april_may_df['destinationAirport'] == 'SFO')].copy()

In [ ]:
# Parse travelDuration
def parse_duration(str_duration):
    '''
        Convert the given string which contains the number of hours and minutes
        to a single float representing the number of hours of the flight.
    '''
    hour = 0
    minute = 0

    # Extract number from string through regular expression
    hour_minute = [(int) (digit) for digit in re.findall(r'\d+', str_duration)]

    if 'DT' in str_duration:
        # If day time is specified
        hour += 24 * hour_minute.pop(0)
        
    if 'H' in str_duration and 'M' in str_duration:
        # Both hour and minute are specified
        hour += hour_minute[0]
        minute = hour_minute[1]
    elif 'H' in str_duration:
        # Only hour is specified
        hour = hour_minute[0]
    else:
        # Only minute is specified
        minute = hour_minute[0]
    
    return hour + minute / 60

# Apply the transformation on new york df and append a new column
new_york_df['parsed_duration'] = new_york_df['travelDuration'].apply(
                                    lambda x : parse_duration(x)
                                 )
new_york_df.drop('travelDuration', axis=1) # drop the old travel duration

# Apply the same transformation on sf df
sf_df['parsed_duration'] = sf_df['travelDuration'].apply(
                                    lambda x : parse_duration(x)
                                 )
sf_df.drop('travelDuration', axis=1) # drop the old travel duration

,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsAirlineCode,segmentsCabinCode,parsed_duration
431,2022-04-16,2022-04-17,ATL,OAK,Y0CXHO,False,False,False,463.58,0,NaN,2022-04-17T21:24:00.000-04:00||2022-04-18T08:45:00.000-07:00,NK||NK,coach||coach,15.833333
432,2022-04-16,2022-04-17,ATL,OAK,Y0CXHO,False,False,False,463.58,0,NaN,2022-04-17T06:35:00.000-04:00||2022-04-17T19:49:00.000-07:00,NK||NK,coach||coach,17.550000
433,2022-04-16,2022-04-17,ATL,OAK,HA0OA0MC,False,False,False,477.61,1,2175.0,2022-04-17T19:36:00.000-04:00||2022-04-17T22:59:00.000-06:00,DL||DL,coach||coach,7.350000
434,2022-04-16,2022-04-17,ATL,OAK,HA0OA0MC,False,False,False,477.61,1,2175.0,2022-04-17T08:10:00.000-04:00||2022-04-17T11:50:00.000-06:00,DL||DL,coach||coach,7.633333
435,2022-04-16,2022-04-17,ATL,OAK,Y0CXHO,False,False,False,510.07,0,NaN,2022-04-17T19:42:00.000-04:00||2022-04-17T22:40:00.000-04:00||2022-04-18T08:45:00.000-07:00,NK||NK||NK,coach||coach||coach,17.533333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5932489,2022-05-06,2022-05-09,JFK,SFO,KH0JUEL1,False,False,True,578.60,7,2566.0,2022-05-09T12:00:00.000-04:00,B6,coach,6.466667
5932490,2022-05-06,2022-05-09,JFK,SFO,YH0JUEY5,False,False,True,713.60,7,2566.0,2022-05-09T06:55:00.000-04:00,AA,coach,6.350000
5932491,2022-05-06,2022-05-09,JFK,SFO,YH0JUEY5,False,False,True,713.60,7,2566.0,2022-05-09T12:00:00.000-04:00,AA,coach,6.466667
5932492,2022-05-06,2022-05-09,JFK,SFO,YH0JUEY5,False,False,True,713.60,7,2566.0,2022-05-09T09:30:00.000-04:00,AA,coach,6.500000


In [ ]:
# aggregate by starting airport and destination airport to 
# calculated mean total price and flight duration 
aggregated_ny_df = new_york_df.groupby(
    by=['startingAirport', 'destinationAirport']
    )[['totalFare', 'parsed_duration']].mean().reset_index()
aggregated_ny_df

,startingAirport,destinationAirport,totalFare,parsed_duration
0,ATL,EWR,243.334567,4.691515
1,ATL,JFK,308.677985,5.483164
2,ATL,LGA,220.564771,4.542456
3,BOS,EWR,177.024419,4.130699
4,BOS,JFK,178.089606,2.229200
5,BOS,LGA,143.624904,3.166926
6,CLT,EWR,252.648955,4.142739
7,CLT,JFK,280.823552,4.965108
8,CLT,LGA,213.972333,3.855167
9,DEN,EWR,348.564513,6.745136


In [ ]:
# aggregate by starting airport and destination airport to 
# calculated mean total price and flight duration 
aggregated_sf_df = sf_df.groupby(
    by=['startingAirport', 'destinationAirport']
    )[['totalFare', 'parsed_duration']].mean().reset_index()
aggregated_sf_df

,startingAirport,destinationAirport,totalFare,parsed_duration
0,ATL,OAK,630.816023,12.152037
1,ATL,SFO,554.468680,10.021577
2,BOS,OAK,601.611038,12.810557
3,BOS,SFO,451.750279,9.654979
4,CLT,OAK,664.895833,12.809369
5,CLT,SFO,577.048896,10.281290
6,DEN,OAK,387.694764,7.889882
7,DEN,SFO,337.759894,5.679192
8,DFW,OAK,480.466269,10.988052
9,DFW,SFO,434.044436,7.467747


In [ ]:
# Plot to see the relationship between travel duration and total fair flying to
# different airports around New York City
alt.Chart(aggregated_ny_df).mark_point().encode(
    y=alt.Y('totalFare', title='Average Total Fare'),
    x='parsed_duration',
    color='destinationAirport',
    facet=alt.Facet('startingAirport:N', columns=5)
).properties(
    width=100,
    height=100,
    title='Travel Duration to Average Total Fare (All flights to NYC)'
)

alt.Chart(...)

In [ ]:
# One interesting thing here is that even though OAK is more eastern, it has longer average flight duraition than SFO.
# try to see if this is due to the lack of direct flight from OAK to EWR/JFK/LGA
oak_total = len(new_york_df[new_york_df['startingAirport'] == 'OAK'])
print('Total flight from OAK to EWR/JFK/LGA): ', oak_total)
oak_non_direct_num = len(new_york_df[(new_york_df['startingAirport'] == 'OAK') & (new_york_df['isNonStop'] == False)])
print('Number of non-direct flight from OAK to EWR/JFK/LGA): ', oak_non_direct_num)
print('Percentage of non-direct flight from OAK to EWR/JFK/LGA: ', oak_non_direct_num / oak_total) 

print()

sfo_total = len(new_york_df[new_york_df['startingAirport'] == 'SFO'])
print('Total flight from SFO to EWR/JFK/LGA): ', sfo_total)
sfo_non_direct_num = len(new_york_df[(new_york_df['startingAirport'] == 'SFO') & (new_york_df['isNonStop'] == False)])
print('Number of non-direct flight from SFO to EWR/JFK/LGA): ', sfo_non_direct_num)
print('Percentage of non-direct flight from SFO to EWR/JFK/LGA: ', sfo_non_direct_num / sfo_total) 

Total flight from OAK to EWR/JFK/LGA):  49063
Number of non-direct flight from OAK to EWR/JFK/LGA):  49063
Percentage of non-direct flight from OAK to EWR/JFK/LGA:  1.0

Total flight from SFO to EWR/JFK/LGA):  92504
Number of non-direct flight from SFO to EWR/JFK/LGA):  65024
Percentage of non-direct flight from SFO to EWR/JFK/LGA:  0.7029317651128599


In [ ]:
# Direct Flight
aggregated_direct_ny_df = new_york_df[new_york_df['isNonStop'] == True].groupby(
    by=['startingAirport', 'destinationAirport']
    )[['totalFare', 'parsed_duration']].mean().reset_index()
aggregated_direct_ny_df

,startingAirport,destinationAirport,totalFare,parsed_duration
0,ATL,EWR,236.953722,2.238933
1,ATL,JFK,237.783008,2.319009
2,ATL,LGA,212.675294,2.302972
3,BOS,EWR,122.537929,1.488253
4,BOS,JFK,140.218420,1.368825
5,BOS,LGA,98.987157,1.403900
6,CLT,EWR,228.132007,1.896095
7,CLT,JFK,249.472971,1.991044
8,CLT,LGA,231.092081,1.814889
9,DEN,EWR,340.406978,3.728489


In [ ]:
alt.Chart(aggregated_direct_ny_df).mark_point().encode(
    y=alt.Y('totalFare', title='Average Total Fare'),
    x='parsed_duration',
    color='destinationAirport',
    facet=alt.Facet('startingAirport:N', columns=5)
).properties(
    width=100,
    height=100,
    title='Travel Duration to Average Total Fare (Direct flights to NYC)'
)

alt.Chart(...)

In [ ]:
# Non Direct flight
aggregated_transfer_ny_df = new_york_df[new_york_df['isNonStop'] == False].groupby(
    by=['startingAirport', 'destinationAirport']
    )[['totalFare', 'parsed_duration']].mean().reset_index()
aggregated_transfer_ny_df

,startingAirport,destinationAirport,totalFare,parsed_duration
0,ATL,EWR,249.848491,7.195248
1,ATL,JFK,354.129617,7.511743
2,ATL,LGA,227.392428,6.480536
3,BOS,EWR,263.105431,8.305392
4,BOS,JFK,392.813183,7.107390
5,BOS,LGA,218.860516,6.138456
6,CLT,EWR,277.693013,6.437684
7,CLT,JFK,299.654491,6.751499
8,CLT,LGA,199.431296,5.588122
9,DEN,EWR,351.343077,7.772645


In [ ]:
alt.Chart(aggregated_transfer_ny_df).mark_point().encode(
    y=alt.Y('totalFare', title='Average Total Fare'),
    x='parsed_duration',
    color='destinationAirport',
    facet=alt.Facet('startingAirport:N', columns=5)
).properties(
    width=100,
    height=100,
    title='Travel Duration to Average Total Fare (Transfer flights to NYC)'
)

alt.Chart(...)

In [ ]:
# Plot to see the relationship between travel duration and total fair flying to
# different airports around San Francisco
alt.Chart(aggregated_sf_df).mark_point().encode(
    y=alt.Y('totalFare', title='Average Total Fare'),
    x='parsed_duration',
    color='destinationAirport',
    facet=alt.Facet('startingAirport:N', columns=4)
).properties(
    width=100,
    height=100,
    title='Travel Duration to Average Total Fare (All flights to SF)'
)

alt.Chart(...)

In [ ]:
# Direct Flight to airports near San Francisco
aggregated_direct_sf_df = sf_df[sf_df['isNonStop'] == True].groupby(
    by=['startingAirport', 'destinationAirport']
    )[['totalFare', 'parsed_duration']].mean().reset_index()
aggregated_direct_sf_df

,startingAirport,destinationAirport,totalFare,parsed_duration
0,ATL,SFO,562.246821,5.226041
1,BOS,SFO,454.970256,6.772966
2,CLT,SFO,624.330142,5.633002
3,DEN,OAK,125.290864,2.618069
4,DEN,SFO,302.294133,2.742472
5,DFW,SFO,421.346723,3.893743
6,DTW,OAK,145.590000,4.933333
7,DTW,SFO,575.072728,5.127699
8,EWR,SFO,420.816529,6.299949
9,IAD,SFO,558.373066,5.932387


In [ ]:
alt.Chart(aggregated_direct_sf_df).mark_point().encode(
    y=alt.Y('totalFare', title='Average Total Fare'),
    x='parsed_duration',
    color='destinationAirport',
    facet=alt.Facet('startingAirport:N', columns=5)
).properties(
    width=100,
    height=100,
    title='Travel Duration to Average Total Fare (Direct flights to SF)'
)

alt.Chart(...)

In [ ]:
# Non Direct flight
aggregated_transfer_sf_df = sf_df[sf_df['isNonStop'] == False].groupby(
    by=['startingAirport', 'destinationAirport']
    )[['totalFare', 'parsed_duration']].mean().reset_index()
aggregated_transfer_sf_df

,startingAirport,destinationAirport,totalFare,parsed_duration
0,ATL,OAK,630.816023,12.152037
1,ATL,SFO,553.122021,10.851847
2,BOS,OAK,601.611038,12.810557
3,BOS,SFO,450.232771,11.013211
4,CLT,OAK,664.895833,12.809369
5,CLT,SFO,573.878701,10.592956
6,DEN,OAK,395.761771,8.051951
7,DEN,SFO,353.309395,6.966758
8,DFW,OAK,480.466269,10.988052
9,DFW,SFO,437.631093,8.477277


In [ ]:
alt.Chart(aggregated_transfer_sf_df).mark_point().encode(
    y=alt.Y('totalFare', title='Average Total Fare'),
    x='parsed_duration',
    color='destinationAirport',
    facet=alt.Facet('startingAirport:N', columns=4)
).properties(
    width=100,
    height=100,
    title='Travel Duration to Average Total Fare (Transfer flights to SF)'
)

alt.Chart(...)

# Inspecting the relationship between price and seat remaining

In [ ]:
# Drop the column search date as one flight can be searched on different dates. Here we only want unique flights.
seats_remain_df = april_may_df.copy().drop(['searchDate'], axis=1).drop_duplicates()
seats_remain_df.head(5)

,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsAirlineCode,segmentsCabinCode
0,2022-04-17,ATL,BOS,LA0NX0MC,PT2H29M,False,False,True,248.6,9,947.0,2022-04-17T12:57:00.000-04:00,DL,coach
1,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,False,False,True,248.6,4,947.0,2022-04-17T06:30:00.000-04:00,DL,coach
2,2022-04-17,ATL,BOS,LA0NX0MC,PT2H30M,False,False,True,248.6,9,947.0,2022-04-17T11:35:00.000-04:00,DL,coach
3,2022-04-17,ATL,BOS,LA0NX0MC,PT2H32M,False,False,True,248.6,8,947.0,2022-04-17T13:59:00.000-04:00,DL,coach
4,2022-04-17,ATL,BOS,LA0NX0MC,PT2H34M,False,False,True,248.6,9,947.0,2022-04-17T09:59:00.000-04:00,DL,coach


In [ ]:
# Prepare columns used in groupby
all_columns = set(seats_remain_df.columns)
exclude_columns = set(['totalFare', 'seatsRemaining'])
all_columns

{'destinationAirport',
 'fareBasisCode',
 'flightDate',
 'isBasicEconomy',
 'isNonStop',
 'isRefundable',
 'seatsRemaining',
 'segmentsAirlineCode',
 'segmentsCabinCode',
 'segmentsDepartureTimeRaw',
 'startingAirport',
 'totalFare',
 'totalTravelDistance',
 'travelDuration'}

In [ ]:
# Group by all columns except total fare and seatsRemaining and count the 
# how many different number of seatsRemaining in each group
seats_count_df = seats_remain_df.groupby(by=list(all_columns - exclude_columns))['seatsRemaining'].count().reset_index()
print(max(seats_count_df['seatsRemaining']))

16


In [ ]:
# Check details of flights that have maximum number of different seatsRemaining
seats_count_df[seats_count_df['seatsRemaining'] == 16]

,segmentsCabinCode,isNonStop,segmentsAirlineCode,destinationAirport,totalTravelDistance,segmentsDepartureTimeRaw,startingAirport,fareBasisCode,isRefundable,travelDuration,isBasicEconomy,flightDate,seatsRemaining
1551556,coach||coach,False,UA||AS,SFO,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,DEN,QAA0OKEN,False,PT5H22M,False,2022-05-07,16
1566079,coach||coach,False,UA||DL,JFK,2501.0,2022-05-11T23:50:00.000-06:00||2022-05-12T11:41:00.000-04:00,DEN,QAA3AKEN,False,PT12H37M,False,2022-05-11,16
1566443,coach||coach,False,UA||DL,JFK,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,DEN,QAA3AKEN,False,PT12H33M,False,2022-05-19,16
1567882,coach||coach,False,UA||DL,LGA,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,IAD,QAA0OFEN,False,PT5H8M,False,2022-05-08,16
2039810,coach||coach||coach,False,UA||AS||AS,OAK,3115.0,2022-05-09T11:00:00.000-04:00||2022-05-09T15:12:00.000-05:00||2022-05-09T20:39:00.000-07:00,LGA,QAA0OHEN,False,PT14H43M,False,2022-05-09,16
2039853,coach||coach||coach,False,UA||AS||AS,OAK,3115.0,2022-05-14T06:30:00.000-04:00||2022-05-14T09:00:00.000-05:00||2022-05-14T14:03:00.000-07:00,LGA,QAA0OHEN,False,PT12H38M,False,2022-05-14,16
2043658,coach||coach||coach,False,UA||DL||DL,OAK,3140.0,2022-05-26T07:00:00.000-04:00||2022-05-26T15:30:00.000-05:00||2022-05-26T18:20:00.000-07:00,LGA,QAA0OKEN,False,PT15H37M,False,2022-05-26,16
2043659,coach||coach||coach,False,UA||DL||DL,OAK,3140.0,2022-05-26T09:50:00.000-04:00||2022-05-26T15:30:00.000-05:00||2022-05-26T18:20:00.000-07:00,LGA,QAA0OKEN,False,PT12H47M,False,2022-05-26,16
2056062,coach||coach||coach,False,UA||UA||DL,OAK,2110.0,2022-05-07T16:55:00.000-04:00||2022-05-07T19:20:00.000-06:00||2022-05-07T22:51:00.000-06:00,DTW,QAA0OFEN,False,PT9H55M,False,2022-05-07,16
2059990,coach||coach||coach,False,UA||UA||DL,OAK,2637.0,2022-05-07T07:02:00.000-04:00||2022-05-07T10:04:00.000-05:00||2022-05-07T16:30:00.000-07:00,CLT,QAA0AFEN,False,PT13H48M,False,2022-05-07,16


In [ ]:
# From original dataframe filter out the flight 1
flight_1 = seats_remain_df[(seats_remain_df['flightDate'] == '2022-05-07') & 
                        (seats_remain_df['startingAirport'] == 'DEN') & 
                        (seats_remain_df['destinationAirport'] == 'SFO') &
                        (seats_remain_df['isNonStop'] == False) &
                        (seats_remain_df['segmentsDepartureTimeRaw'] == 
                         '2022-05-07T13:26:00.000-06:00||' +
                          '2022-05-07T15:45:00.000-07:00') & 
                        (seats_remain_df['travelDuration'] == 'PT5H22M') &
                        (seats_remain_df['segmentsAirlineCode'] == 'UA||AS') & 
                        (seats_remain_df['fareBasisCode'] == 'QAA0OKEN')]
flight_1

,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsAirlineCode,segmentsCabinCode
261943,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,421.60,1,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
649176,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,421.60,2,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
1023079,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,421.60,3,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
2046395,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,451.60,4,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
2713419,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,426.60,3,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
2938904,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,371.60,6,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
3254231,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,347.98,1,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
3543684,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,430.60,1,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
3840249,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,436.59,1,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach
4128330,2022-05-07,DEN,SFO,QAA0OKEN,PT5H22M,False,False,False,451.60,2,1242.0,2022-05-07T13:26:00.000-06:00||2022-05-07T15:45:00.000-07:00,UA||AS,coach||coach


In [ ]:
# Plot to see the avergae price for each number of seats remaining for flight 1
alt.Chart(flight_1).mark_point().encode(
    y='mean(totalFare)',
    x='seatsRemaining',
).properties(
    title='Seats Remaining to Average Prices (Flight 1)'
)

alt.Chart(...)

In [ ]:
# From original dataframe filter out the flight 2
flight_2 = seats_remain_df[
    (seats_remain_df['flightDate'] == '2022-05-19') & 
    (seats_remain_df['startingAirport'] == 'DEN') & 
    (seats_remain_df['destinationAirport'] == 'JFK') &
    (seats_remain_df['isNonStop'] == False) &
    (seats_remain_df['segmentsDepartureTimeRaw'] == 
     '2022-05-19T08:26:00.000-06:00||' + 
     '2022-05-19T19:58:00.000-04:00') & 
     (seats_remain_df['travelDuration'] == 'PT12H33M') &
     (seats_remain_df['segmentsAirlineCode'] == 'UA||DL') & 
     (seats_remain_df['fareBasisCode'] == 'QAA3AKEN')]
flight_2

,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsAirlineCode,segmentsCabinCode
370738,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,547.20,5,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
756046,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,547.20,2,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
1485430,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,521.21,6,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
1811956,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,521.21,8,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
2152841,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,521.21,9,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
2486586,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,521.21,7,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
2820448,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,521.21,3,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
3359105,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,521.21,4,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
3652224,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,531.20,4,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach
3948453,2022-05-19,DEN,JFK,QAA3AKEN,PT12H33M,False,False,False,557.21,5,2794.0,2022-05-19T08:26:00.000-06:00||2022-05-19T19:58:00.000-04:00,UA||DL,coach||coach


In [ ]:
# Plot to see the avergae price for each number of seats remaining for flight 2
alt.Chart(flight_2).mark_point().encode(
    y='mean(totalFare)',
    x='seatsRemaining',
).properties(
    title='Seats Remaining to Average Prices (Flight 2)'
)

alt.Chart(...)

In [ ]:
# From original dataframe filter out the flight 3
flight_3 = seats_remain_df[
    (seats_remain_df['flightDate'] == '2022-05-08') & 
    (seats_remain_df['startingAirport'] == 'IAD') & 
    (seats_remain_df['destinationAirport'] == 'LGA') &
    (seats_remain_df['isNonStop'] == False) &
    (seats_remain_df['segmentsDepartureTimeRaw'] == 
     '2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00') & 
    (seats_remain_df['travelDuration'] == 'PT5H8M') &
    (seats_remain_df['segmentsAirlineCode'] == 'UA||DL') & 
    (seats_remain_df['fareBasisCode'] == 'QAA0OFEN')]
flight_3

,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsAirlineCode,segmentsCabinCode
659373,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,461.6,5,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
1033442,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,461.6,6,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
1391207,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,461.6,7,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
1716639,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,461.6,8,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
2723796,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,461.6,4,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
2949584,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,491.6,7,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
3265000,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,491.6,6,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
3554301,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,531.6,4,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
3850824,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,531.6,5,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach
4138903,2022-05-08,IAD,LGA,QAA0OFEN,PT5H8M,False,False,False,571.6,6,710.0,2022-05-08T08:21:00.000-04:00||2022-05-08T11:52:00.000-04:00,UA||DL,coach||coach


In [ ]:
# Plot to see the avergae price for each number of seats remaining for flight 3
alt.Chart(flight_3).mark_point().encode(
    y='mean(totalFare)',
    x='seatsRemaining',
).properties(
    title='Seats Remaining to Average Prices (Flight 3)'
)

alt.Chart(...)

## Get the data for the ful time duration to see a bigger picture of the data.

In [ ]:
alltime_df = pd.read_csv('drive/My Drive/Data_Science_Big_Data_T11/Alltime_Data.csv')
alltime_df.tail()

,searchDate,flightDate,totalFare
82138748,2022-10-05,2022-11-12,224.60
82138749,2022-10-05,2022-11-12,228.59
82138750,2022-10-05,2022-11-12,251.60
82138751,2022-10-05,2022-11-12,251.60
82138752,2022-10-05,2022-11-12,268.60


In [ ]:
alltime_df['flightDate'] = pd.to_datetime(alltime_df['flightDate'])

alldate_to_fare = pd.DataFrame(alltime_df.groupby('flightDate')['totalFare'].mean().reset_index())
searchalldate_to_fare = pd.DataFrame(alltime_df.groupby('searchDate')['totalFare'].mean().reset_index())

searchalldate_to_fare.head()

,searchDate,totalFare
0,2022-04-16,371.575580
1,2022-04-17,349.504482
2,2022-04-18,344.606550
3,2022-04-19,359.993897
4,2022-04-20,349.483219


# Exploratory Analysis

In [ ]:
alltime_df.totalFare.describe()

In [ ]:
alldate_to_fare.totalFare.describe()

NameError: ignored

In [ ]:
searchalldate_to_fare.totalFare.describe()

In [ ]:
interval = alt.selection_interval(encodings=['x'])

base = alt.Chart(alldate_to_fare).mark_line().encode(
    x='flightDate:T',
    y='totalFare:Q',
    tooltip=[
        alt.Tooltip('flightDate:T', title='Flight Date'),
        alt.Tooltip('totalFare:Q', title='Total Fare')
    ]
).properties(
    title = 'Daily Mean Flight Price from April to November'
)

chart = base.properties(
    width=800,
    height=300
).encode(
    x = alt.X('flightDate:T', scale=alt.Scale(domain=interval.ref()))
)

view = chart.properties(
    width=800,
    height=50,
    selection = interval
)

chart & view

alt.VConcatChart(...)

In [ ]:
interval = alt.selection_interval(encodings=['x'])

search = alt.Chart(searchalldate_to_fare).mark_line().encode(
    x='searchDate:T',
    y='totalFare:Q',
    tooltip=[
        alt.Tooltip('searchDate:T', title='Search Date'),
        alt.Tooltip('totalFare:Q', title='Total Fare')
    ]
).properties(
    title = 'Daily Average for Searching Flight Price from April to November'
)

chart = search.properties(
    width=800,
    height=300
).encode(
    x = alt.X('searchDate:T', scale=alt.Scale(domain=interval.ref()))
)

view = chart.properties(
    width=800,
    height=50,
    selection = interval
)

chart & view

alt.VConcatChart(...)

In [ ]:
base = alt.Chart(alldate_to_fare).mark_bar().encode(
    x = alt.X('flightDate:O', timeUnit='month'),
    y = 'mean(totalFare):Q'
)
search = alt.Chart(searchalldate_to_fare).mark_bar(color='red', opacity=0.5).encode(
    x = alt.X('searchDate:O', timeUnit='month'),
    y = 'mean(totalFare):Q'
).properties(
    title = 'Mean Flight Price by month'
)

(base + search).properties(title='Average Price by Month')

alt.LayerChart(...)

In [ ]:
alt.Chart(alldate_to_fare).mark_rect().encode(
    x = alt.X('flightDate:O', timeUnit = 'date'),
    y = alt.Y('flightDate:O', timeUnit = 'month'),
    color = alt.Color('mean(totalFare):Q', scale=alt.Scale(scheme='greens')),
    tooltip=[
        alt.Tooltip('flightDate:T', title='Flight Date'),
        alt.Tooltip('totalFare:Q', title='Total Fare')
    ]
).properties(
    title = 'Mean Flight Price by every days in months'
)

alt.Chart(...)

In [ ]:
alt.Chart(alldate_to_fare).mark_rect().encode(
    x = alt.X('flightDate:O', timeUnit = 'week'),
    y = alt.Y('flightDate:O', timeUnit = 'month'),
    color = alt.Color('mean(totalFare):Q', scale=alt.Scale(scheme='greens')),
    tooltip=[
        alt.Tooltip('flightDate:T', title='Flight Date'),
        alt.Tooltip('totalFare:Q', title='Total Fare')
    ]
).properties(
    title = 'Mean Flight Price by weeks from April to November'
)

alt.Chart(...)